# Modèles en local

## Prétraitement

Dans ce notebook, nous allons préparer notre dataframe afin qu'il soit utilisable par des modèles de machine learning.

In [ ]:
import polars as pl

df = pl.read_parquet("dataframes/eda.parquet")
df

Nous allons commencer par mettre en place une fonction de nettoyage de nos textes.

In [ ]:
import re
import numpy as np


def clean_text(batch):

	cleaned = []
	for row in batch:
		row = re.sub(r'https?://\S+|www\.\S+', ' ', row) # URLs
		row = re.sub(r'@\w+', ' ', row)	# mentions
		row = re.sub(r'#(\w+)', r'\1', row) # hashtags
		row = re.sub(r'\s+', ' ', row).strip() # espaces
		cleaned.append(row)

	return np.array(cleaned)

## Régression logistique

Ensuite, mettons en place le pipeline du modèle. Nous allons y mettre notre fonction de nettoyage du texte, notre vectorizer et enfin le modèle.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


preprocess_and_model = Pipeline([
	("cleaner", FunctionTransformer(clean_text)),
	("tfidf", TfidfVectorizer(
		min_df=2,
		max_df=0.9,
		ngram_range=(1, 2),
	)),
	("clf", LogisticRegression(max_iter=200))
])

Maintenant séparons nos données d'entraînement et nos données de validation, et enfin testons notre modèle dans un process MLFlow.

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
import joblib


X_train, X_test, y_train, y_test = train_test_split(
    df["content"].to_list(),
    df["target"].to_list(),
    test_size=0.2,
    random_state=0,
    stratify=df["target"]
)

MODEL_PATH = "models/logistic_regression.joblib"

try:

	lr_model = joblib.load(MODEL_PATH)
	test_score = lr_model.score(X_test, y_test)

except FileNotFoundError:
    
	experiment_name = "Réalisez une analyse de sentiments grâce au Deep Learning"
	mlflow.set_experiment(experiment_name=experiment_name)

	with mlflow.start_run(run_name="logistic_regression"):

		mlflow.sklearn.autolog() # type: ignore

		preprocess_and_model.fit(X_train, y_train)

		train_score = preprocess_and_model.score(X_train, y_train)
		test_score = preprocess_and_model.score(X_test, y_test)

		mlflow.log_metric("train_score", float(train_score))
		mlflow.log_metric("test_score", float(test_score))

		joblib.dump(preprocess_and_model, MODEL_PATH)
		lr_model = preprocess_and_model

print("Score:", test_score)

On obtient un bon très bon score, surtout pour un modèle si basique. Cela nous rassure sur la faisabilité du projet.

Testons une phrase personnalisée.

In [ ]:
print(lr_model.predict_proba(["Im not happy"]))

Notre modèle comprend bien que "not happy" est probablement négatif.

In [ ]:
df = df.with_columns(pl.col("content").map_batches(clean_text).alias("clean_content"))

df.write_parquet("dataframes/preprocessed.parquet")